In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt 
import distance
import json
import os
import slack
from datetime import timedelta,datetime
from sshtunnel import SSHTunnelForwarder
import psycopg2
import nbslack

In [ ]:
webhook = 'https://hooks.slack.com/services/' + str(os.environ.get('SLACK_ANALYTICS_WEB'))
nbslack.notifying('Siddhesvar',webhook)

In [ ]:
try:
    with SSHTunnelForwarder(
        (os.environ.get('JOUST_IP'), 22),
        ssh_private_key="~/.ssh/id_rsa",
        ssh_username="ubuntu",
        remote_bind_address=(os.environ.get('JOUST_ADDRESS'), 5432)) as server:

        server.start()
        print("server connected")

        params = {
            "database": "joust_production",
            "user": "sid",
            "password": os.environ.get('JOUST_PASS'),
            "host": "localhost",
            "port": server.local_bind_port
        }

        conn = psycopg2.connect(**params)
        curs = conn.cursor()
        print("database connected")

        #curs.execute("select * from public.account_holders")
        total = "select account_holders.id, first_name, last_name, occupations.\"name\" as occupation, industries.\"name\" as industry, type_of_work, account_holders.created_at, marketing.value as legacy_business_url, email, date_of_birth, q2_core_pro_customer_id from account_holders left join occupations on account_holders.occupation_id = occupations.id left join industries on occupations.industry_id = industries.id left join businesses on account_holders.id = businesses.account_holder_id left join marketing on businesses.id = marketing.business_id"
                
        #result = curs.fetchall()
        tra = pd.read_sql_query(transfers, conn)
        ids = pd.read_sql_query(total, conn)
        req = pd.read_sql_query(invoices, conn)

        conn.close()

except Exception as e:
    print(e)
    print("Connection Failed")

In [ ]:
fin = ids
fin['created_at'] = pd.to_datetime(fin['created_at'])
fin['created_at'] = fin['created_at'].dt.tz_localize('utc')
fin['created_at'] = fin['created_at'].dt.tz_convert('US/Central')

In [ ]:
yest = datetime.utcnow() - timedelta(1)
yr = yest.year
mo = yest.month
dy = yest.day

In [ ]:
fin['Year'] = fin['created_at'].dt.year
fin['Month'] = fin['created_at'].dt.month
fin['Day'] = fin['created_at'].dt.day

In [ ]:
fin2 = fin[fin['Year'] == yr]
fin2 = fin2[fin2['Month'] == mo]
fin2 = fin2[fin2['Day'] == dy]

In [ ]:
badwords = ['blood','weed','drugs']

In [ ]:
emails = fin2[['email']].dropna()['email'].values
urls = fin2[['legacy_business_url']].dropna()['legacy_business_url'].values
jobs = fin2[['occupation']].dropna()['occupation'].values

In [ ]:
emails2 = []
urls2 = []
jobs2 = []
for b in badwords:
    emails3 = [i for i in emails if b in i]
    emails2.extend(emails3)
    urls3 = [i for i in urls if b in i]
    urls2.extend(urls3)
    jobs3 = [i for i in jobs if b in i]
    jobs2.extend(jobs3)

In [ ]:
ids = []
for z in emails2:
    ids.extend(fin2[fin2['email'] == z]['id'].values)
for z in urls2:
    ids.extend(fin2[fin2['legacy_business_url'] == z]['id'].values)
for z in jobs2:
    ids.extend(fin2[fin2['occupation'] == z]['id'].values)

In [ ]:
str(ids)

In [ ]:
0 * * * * eval `ssh-agent -s` && ssh-add && //anaconda3/bin/python /Users/siddhesvark/Desktop/Data\ Analysis/Deployed\ MAU/Daily\ Slack\ Info.py
chmod +x /Users/siddhesvark/Desktop/Data\ Analysis/Deployed\ MAU/Daily\ Slack\ Info.py